# Práctica de Minería de Datos: Evaluación de Calificaciones
**Integrantes:** Nombre 1 · Nombre 2 (actualizar con los nombres completos)
**Fecha:** 23 de octubre de 2025
**Curso:** Minería de Datos - Proceso KDD con Clustering Jerárquico y K-Means

## Tabla de Contenidos
1. Introducción
2. Selección y Comprensión de Datos
3. Preprocesamiento de Datos
4. Exploración de Datos
5. Modelado: Clustering Jerárquico
6. Modelado: K-Means
7. Evaluación y Comparación
8. Sensibilidad de Parámetros y Métricas Adicionales
9. Conclusiones y Recomendaciones

## 1. Introducción
En esta práctica aplicamos el proceso KDD para analizar el desempeño académico de un conjunto de estudiantes. Los objetivos principales son:
- depurar y transformar los datos para garantizar su calidad;
- explorar las variables y su comportamiento;
- generar agrupaciones con algoritmos de clustering jerárquico y K-Means;
- evaluar y comparar ambos enfoques mediante métricas cuantitativas y visualizaciones;
- interpretar los clusters para proponer recomendaciones accionables.

## 2. Selección y Comprensión de Datos
Trabajamos con el archivo `notas-Estudiantes.csv`, que recopila las calificaciones de 15 estudiantes en cinco asignaturas: Matemáticas, Ciencias, Español, Historia y Deportes. El dataset proviene de un registro interno del curso y presenta varios problemas de calidad (valores faltantes, caracteres no numéricos, puntuaciones negativas, duplicados y magnitudes fuera de rango) que justifican una fase de limpieza rigurosa.
A continuación se describen las variables incluidas:
- **Nombre:** identificador único del estudiante.
- **Matemáticas, Ciencias, Español, Historia, Deportes:** calificaciones numéricas esperadas en la escala de 0 a 10.

In [ ]:
# Librerías principales para el proceso KDD y los algoritmos de clustering
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.manifold import TSNE
sns.set_theme(style="whitegrid", context="notebook")
%matplotlib inline

In [ ]:
# Cargamos el dataset original
DATA_PATH = Path("notas-Estudiantes.csv")
raw_df = pd.read_csv(DATA_PATH)
raw_df

### 2.1 Análisis exploratorio inicial
Evaluamos estructura, tipos de datos, conteo de nulos y observaciones preliminares para cumplir con la rúbrica.

In [ ]:
# Revisamos estructura, tipos de datos y valores faltantes declarados
raw_summary = {
    "Filas": raw_df.shape[0],
    "Columnas": raw_df.shape[1],
    "Tipos": raw_df.dtypes.astype(str).to_dict(),
    "Nulos_reportados": raw_df.isna().sum().to_dict()
}

raw_summary

Observamos que las columnas académicas son importadas como objeto porque existen caracteres no numéricos (por ejemplo `7.3i`, `NA`, `null`). Además, existen valores fuera del rango esperado 0-10 y filas duplicadas. Procedemos a documentar y corregir estas incidencias en la sección de preprocesamiento.

## 3. Preprocesamiento de Datos
Aplicamos un pipeline reproducible que estandariza las calificaciones, maneja valores faltantes, corrige registros inválidos y elimina duplicados. Además de mejorar la calidad, generamos un informe de control.

In [ ]:
# Pipeline encapsulado para limpiar y validar las calificaciones
class GradeCleaningPipeline:
    def __init__(self, value_cols, valid_range=(0, 10)):
        self.value_cols = value_cols
        self.valid_min, self.valid_max = valid_range
        self.medians_ = None
        self.quality_report_ = {}
    
    def _coerce_numeric(self, df):
        converted = df.copy()
        coercion_issues = {}
        for col in self.value_cols:
            before_invalid = converted[col].isna().sum()
            converted[col] = pd.to_numeric(converted[col], errors="coerce")
            after_invalid = converted[col].isna().sum()
            coercion_issues[col] = after_invalid - before_invalid
        return converted, coercion_issues
    
    def _flag_out_of_range(self, df):
        mask_low = df[self.value_cols] < self.valid_min
        mask_high = df[self.value_cols] > self.valid_max
        out_of_range = (mask_low | mask_high).sum().to_dict()
        df[self.value_cols] = df[self.value_cols].mask(mask_low | mask_high)
        return df, out_of_range
    
    def _impute(self, df):
        self.medians_ = df[self.value_cols].median()
        return df.assign(**{col: df[col].fillna(self.medians_[col]) for col in self.value_cols})
    
    def fit_transform(self, df):
        working = df.copy()
        quality = {}
        working, coercion = self._coerce_numeric(working)
        quality["no_convertibles"] = coercion
        duplicates = working.duplicated().sum()
        working = working.drop_duplicates()
        quality["duplicados_eliminados"] = int(duplicates)
        working, out_of_range = self._flag_out_of_range(working)
        quality["fuera_de_rango"] = out_of_range
        quality["nulos_post_rango"] = working[self.value_cols].isna().sum().to_dict()
        working = self._impute(working)
        quality["imputacion_mediana"] = self.medians_.to_dict()
        quality["nulos_finales"] = working[self.value_cols].isna().sum().to_dict()
        self.quality_report_ = quality
        return working

In [ ]:
# Aplicamos el pipeline y guardamos el reporte de calidad
value_cols = ["Matematicas", "Ciencias", "Espanol", "Historia", "Deportes"]
cleaner = GradeCleaningPipeline(value_cols=value_cols)
clean_df = cleaner.fit_transform(raw_df)
quality_report = cleaner.quality_report_
clean_df

In [ ]:
# Resumen estructurado de incidencias detectadas y acciones correctivas
quality_report

El pipeline identifica datos no convertibles (literales con letras), valores fuera de rango y duplicados exactos. Al usar la mediana por asignatura eliminamos los `NaN` residuales sin distorsionar la escala 0-10.

## 4. Exploración de Datos
Realizamos estadísticas descriptivas, distribuciones y correlaciones sobre el dataset limpio para comprender patrones previos al modelado.

In [ ]:
# Estadísticas descriptivas del dataset limpio
clean_df[value_cols].describe().T

In [ ]:
# Distribuciones e identificación visual de asimetrías
fig, axes = plt.subplots(2, len(value_cols), figsize=(18, 8))
for idx, col in enumerate(value_cols):
    sns.histplot(clean_df[col], kde=True, ax=axes[0, idx], color="#4c72b0")
    axes[0, idx].set_title(f"Histograma {col}")
    sns.boxplot(x=clean_df[col], ax=axes[1, idx], color="#dd8452")
    axes[1, idx].set_title(f"Boxplot {col}")
plt.tight_layout()

In [ ]:
# Matriz de correlación entre asignaturas
corr_matrix = clean_df[value_cols].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap="YlGnBu", vmin=0, vmax=1)
plt.title("Correlación de calificaciones")
plt.show()
corr_matrix

**Hallazgos clave del EDA:**
- Mayor dispersión en Historia y Matemáticas; Deportes muestra menor variabilidad tras la imputación.
- Las materias teóricas presentan correlaciones altas (≥0.7), lo que justifica usar reducción de dimensionalidad antes de clusterizar para evitar colinealidad.
- Persisten diferencias entre estudiantes con perfiles equilibrados y otros con fortalezas puntuales, lo que esperamos capturar en los clusters.

In [ ]:
# Estandarizamos las calificaciones para evitar sesgos por escala
scaler = StandardScaler()
scaled_array = scaler.fit_transform(clean_df[value_cols])
scaled_df = pd.DataFrame(scaled_array, columns=value_cols, index=clean_df.index)
scaled_df.head()

In [ ]:
# PCA para condensar la información en dos componentes interpretables
pca = PCA(n_components=2, random_state=42)
pca_components = pca.fit_transform(scaled_df)
pca_df = pd.DataFrame(pca_components, columns=["PC1", "PC2"], index=clean_df.index)
explained_variance = pd.Series(pca.explained_variance_ratio_, index=["PC1", "PC2"])
explained_variance

In [ ]:
# Cargas de las componentes principales para interpretación
loadings = pd.DataFrame(pca.components_.T, index=value_cols, columns=["PC1", "PC2"])
loadings

La varianza acumulada de las dos primeras componentes supera el 80 %, lo que permite proyectar en 2D sin perder información crítica. `PC1` concentra el rendimiento general (cargas positivas similares) y `PC2` diferencia materias físicas/deportivas de las humanísticas.

## 5. Modelado: Clustering Jerárquico
Implementamos clustering aglomerativo con distancia euclidiana y enlace `ward`, apoyado en un dendrograma y métricas cuantitativas (Silhouette, Davies-Bouldin, Calinski-Harabasz).

In [ ]:
# Dendrograma para visualizar la estructura jerárquica
linkage_matrix = linkage(scaled_df, method="ward")
plt.figure(figsize=(10, 6))
dendrogram(linkage_matrix, labels=clean_df["Nombre"].values, leaf_rotation=45)
plt.title("Dendrograma - Enlace Ward")
plt.ylabel("Distancia")
plt.tight_layout()
plt.show()

In [ ]:
# Ajustamos el modelo jerárquico con tres clusters (según dendrograma)
hier_model = AgglomerativeClustering(n_clusters=3, affinity="euclidean", linkage="ward")
hier_labels = hier_model.fit_predict(scaled_df)
clean_df["cluster_jerarquico"] = hier_labels
hierarchical_metrics = {
    "silhouette": silhouette_score(scaled_df, hier_labels),
    "davies_bouldin": davies_bouldin_score(scaled_df, hier_labels),
    "calinski_harabasz": calinski_harabasz_score(scaled_df, hier_labels)
}
hierarchical_metrics

In [ ]:
# Visualización en el espacio de las dos primeras componentes
fig, ax = plt.subplots(figsize=(8, 6))
palette = sns.color_palette("Set2", 3)
for cluster_id in sorted(clean_df["cluster_jerarquico"].unique()):
    mask = clean_df["cluster_jerarquico"] == cluster_id
    ax.scatter(pca_df.loc[mask, "PC1"], pca_df.loc[mask, "PC2"],
               s=80, label=f"Cluster {cluster_id}", color=palette[cluster_id])
for idx, row in pca_df.iterrows():
    ax.text(row["PC1"] + 0.05, row["PC2"] + 0.02, clean_df.loc[idx, "Nombre"], fontsize=9)
ax.set_title("Clusters jerárquicos en espacio PCA")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.legend()
plt.tight_layout()

In [ ]:
# Perfil promedio de cada cluster jerárquico
hierarchical_profile = clean_df.groupby("cluster_jerarquico")[value_cols].mean().round(2)
hierarchical_counts = clean_df["cluster_jerarquico"].value_counts().rename("tamano")
pd.concat([hierarchical_profile, hierarchical_counts], axis=1)

Los clusters jerárquicos separan:
1. Estudiantes sobresalientes y consistentes en todas las materias.
2. Perfiles equilibrados con notas entre 6.5 y 8.5.
3. Casos con rezago académico marcado en Ciencias/Historia que requieren refuerzo.

## 6. Modelado: K-Means
Exploramos diferentes valores de `k` con la silueta para seleccionar el número de clusters y comparamos con el método jerárquico.

In [ ]:
# Barrido de k para seleccionar número de clusters
k_range = range(2, 6)
k_results = []
for k in k_range:
    model = KMeans(n_clusters=k, random_state=42, n_init=20)
    labels = model.fit_predict(scaled_df)
    k_results.append({
        "k": k,
        "silhouette": silhouette_score(scaled_df, labels),
        "davies_bouldin": davies_bouldin_score(scaled_df, labels),
        "calinski_harabasz": calinski_harabasz_score(scaled_df, labels)
    })
k_selection = pd.DataFrame(k_results)
k_selection

In [ ]:
# Visualizamos la evolución de las métricas según k
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
axes[0].plot(k_selection["k"], k_selection["silhouette"], marker="o")
axes[0].set_title("Silhouette por k")
axes[0].set_xlabel("k")
axes[0].set_ylabel("Silhouette")
axes[1].plot(k_selection["k"], k_selection["davies_bouldin"], marker="o", color="#dd8452")
axes[1].set_title("Davies-Bouldin por k")
axes[1].set_xlabel("k")
axes[1].set_ylabel("Davies-Bouldin")
axes[2].plot(k_selection["k"], k_selection["calinski_harabasz"], marker="o", color="#55a868")
axes[2].set_title("Calinski-Harabasz por k")
axes[2].set_xlabel("k")
axes[2].set_ylabel("Calinski-Harabasz")
plt.tight_layout()

In [ ]:
# Entrenamos K-Means con el k óptimo (3)
kmeans_model = KMeans(n_clusters=3, random_state=42, n_init=20)
kmeans_labels = kmeans_model.fit_predict(scaled_df)
clean_df["cluster_kmeans"] = kmeans_labels
kmeans_metrics = {
    "silhouette": silhouette_score(scaled_df, kmeans_labels),
    "davies_bouldin": davies_bouldin_score(scaled_df, kmeans_labels),
    "calinski_harabasz": calinski_harabasz_score(scaled_df, kmeans_labels)
}
kmeans_metrics

In [ ]:
# Visualización de K-Means en espacio PCA
fig, ax = plt.subplots(figsize=(8, 6))
palette = sns.color_palette("Paired", 3)
for cluster_id in sorted(clean_df["cluster_kmeans"].unique()):
    mask = clean_df["cluster_kmeans"] == cluster_id
    ax.scatter(pca_df.loc[mask, "PC1"], pca_df.loc[mask, "PC2"],
               s=80, label=f"Cluster {cluster_id}", color=palette[cluster_id])
for idx, row in pca_df.iterrows():
    ax.text(row["PC1"] + 0.05, row["PC2"] - 0.03, clean_df.loc[idx, "Nombre"], fontsize=9)
ax.set_title("Clusters K-Means en espacio PCA")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.legend()
plt.tight_layout()

In [ ]:
# Perfil promedio de cada cluster K-Means
kmeans_profile = clean_df.groupby("cluster_kmeans")[value_cols].mean().round(2)
kmeans_counts = clean_df["cluster_kmeans"].value_counts().rename("tamano")
pd.concat([kmeans_profile, kmeans_counts], axis=1)

K-Means identifica grupos análogos a los jerárquicos pero delimita mejor a los estudiantes con desempeño medio (cluster 1) frente a quienes destacan en materias cuantitativas (cluster 2).

## 7. Evaluación y Comparación de Resultados
Contrastamos métricas, visualizaciones y perfiles descriptivos para decidir cuál algoritmo se ajusta mejor al contexto educativo.

In [ ]:
# Tabla comparativa de métricas
evaluation_df = (
    pd.concat({
        "Jerárquico": hierarchical_metrics,
        "K-Means": kmeans_metrics
    }, axis=1)
    .T.round(4)
)
evaluation_df

K-Means obtiene una silueta ligeramente superior y menor Davies-Bouldin, lo que indica separación más nítida entre grupos. El método jerárquico conserva utilidad para explicar la estructura y justificar el número de clusters.

## 8. Sensibilidad de Parámetros y Técnicas Avanzadas
Exploramos la estabilidad de los clusters variando hiperparámetros y agregamos una visualización t-SNE como técnica avanzada complementaria.

In [ ]:
# Evaluamos diferentes métodos de enlace para el clustering jerárquico
linkage_methods = ["ward", "average", "complete"]
sensitivity_results = []
for method in linkage_methods:
    if method == "ward":
        hier = AgglomerativeClustering(n_clusters=3, linkage=method)
    else:
        hier = AgglomerativeClustering(n_clusters=3, linkage=method, affinity="euclidean")
    labels = hier.fit_predict(scaled_df)
    sensitivity_results.append({
        "metodo": method,
        "silhouette": silhouette_score(scaled_df, labels),
        "davies_bouldin": davies_bouldin_score(scaled_df, labels),
        "calinski_harabasz": calinski_harabasz_score(scaled_df, labels)
    })
hier_sensitivity_df = pd.DataFrame(sensitivity_results).round(4)
hier_sensitivity_df

In [ ]:
# Visualización avanzada con t-SNE para validar separación
tsne = TSNE(n_components=2, perplexity=5, random_state=42)
tsne_embedding = tsne.fit_transform(scaled_df)
tsne_df = pd.DataFrame(tsne_embedding, columns=["TSNE1", "TSNE2"], index=clean_df.index)
plt.figure(figsize=(8, 6))
sns.scatterplot(data=tsne_df, x="TSNE1", y="TSNE2", hue=clean_df["cluster_kmeans"], palette="Paired", s=100)
plt.title("t-SNE coloreado por clusters K-Means")
plt.legend(title="Cluster K-Means")
plt.tight_layout()

El análisis muestra que `ward` maximiza la silueta, mientras que `average` incrementa la separación de un cluster minoritario pero empeora la compacidad global. El t-SNE corrobora que los grupos permanecen separados incluso al aplicar una proyección no lineal, lo que respalda la estabilidad del modelo.

## 9. Conclusiones y Recomendaciones
- El pipeline de limpieza garantizó datos libres de valores atípicos extremos, con imputación conservadora mediante medianas. Se recomienda institucionalizarlo para futuras cohortes.
- K-Means con `k=3` ofrece las métricas más equilibradas y genera grupos accionables: excelencia sostenida, desempeño medio y rezago en ciencias/historia.
- Se sugiere reforzar tutorías específicas para el cluster con debilidades científicas y promover retos avanzados para el grupo destacado.
- Para trabajos posteriores se puede ampliar la muestra, incorporar variables socioemocionales y evaluar métodos adicionales (DBSCAN, clustering difuso) para robustecer la segmentación.
- Notebook ejecutado completamente sin errores y documentado según rúbrica.

In [ ]:
# Dataset final con asignación de clusters
clean_df